In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
from imblearn.over_sampling import SMOTE

In [2]:
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data2/Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

In [3]:
#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

Count= int(322/10)*83

In [4]:
for i in range(10):
    #마지막 그룹은 34명
    if i==9:
        X_test=X.iloc[Count*i:-1]
        X_train=X.drop(X.index[Count*i:-1])
        y_test=y.iloc[Count*i:-1]
        y_train=y.drop(y.index[Count*i:-1])
    
    #모든 그룹은 32명씩 (마지막은 34)
    X_test=X.iloc[Count*i:Count*(i+1)]
    X_train=X.drop(X.index[Count*i:Count*(i+1)])
    y_test=y.iloc[Count*i:Count*(i+1)]
    y_train=y.drop(y.index[Count*i:Count*(i+1)])
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train,y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    
    #모델 정의, 예측
    model= LinearDiscriminantAnalysis().fit(X_train,y_train)
    
    predict=model.predict(X_test)
    
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))
    #f1score
    f1 = f1_score(y_test,predict,pos_label='positive', average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print()
    
    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=f1
    Result[i][2]=list[0]
    Result[i][3]=list[1]

[0]Accuracy : 0.5055578427336352
[0]F1score : 0.49858542265165817
[0]Precision : 0.506620144680236
[0]Recall : 0.5055578427336352

[1]Accuracy : 0.36319444444444443
[1]F1score : 0.3500438992318502
[1]Precision : 0.35405419745297145
[1]Recall : 0.36319444444444443

[2]Accuracy : 0.4748803827751196
[2]F1score : 0.4762989112161901
[2]Precision : 0.504939555832431
[2]Recall : 0.4748803827751196



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


[3]Accuracy : 0.44373503591380686
[3]F1score : 0.4408506668046466
[3]Precision : 0.45497255811673887
[3]Recall : 0.44373503591380686

[4]Accuracy : 0.41630020521841105
[4]F1score : 0.41887169119132817
[4]Precision : 0.42334173279772597
[4]Recall : 0.41630020521841105

[5]Accuracy : 0.3285965624577074
[5]F1score : 0.3005543569542089
[5]Precision : 0.32664922094302884
[5]Recall : 0.3285965624577074



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


[6]Accuracy : 0.3918461330082622
[6]F1score : 0.38986325914698544
[6]Precision : 0.3895396263940439
[6]Recall : 0.3918461330082622

[7]Accuracy : 0.5073684210526316
[7]F1score : 0.4763051572406058
[7]Precision : 0.5181863162320126
[7]Recall : 0.5073684210526316

[8]Accuracy : 0.35084882780921584
[8]F1score : 0.3214309162224726
[8]Precision : 0.3257054764459058
[8]Recall : 0.35084882780921584



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


[9]Accuracy : 0.3744418177042291
[9]F1score : 0.35070278400642985
[9]Precision : 0.3436041839093566
[9]Recall : 0.3744418177042291



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1386: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [5]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.505558,0.498585,0.506620,0.505558
1,0.363194,0.350044,0.354054,0.363194
2,0.474880,0.476299,0.504940,0.474880
3,0.443735,0.440851,0.454973,0.443735
4,0.416300,0.418872,0.423342,0.416300
5,0.328597,0.300554,0.326649,0.328597
6,0.391846,0.389863,0.389540,0.391846
7,0.507368,0.476305,0.518186,0.507368
8,0.350849,0.321431,0.325705,0.350849
9,0.374442,0.350703,0.343604,0.374442


In [6]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.4156769673117463
Average of F1-Score 0.40235070646663756
Average of Precision 0.4147613012804451
Average of Recall 0.4156769673117463
